# Link Analysis Notebook
By Raihaan Usman

### Initialization

In [1]:
import numpy as np
from numpy import sin, cos, tan, pi, sqrt
from scipy.optimize import minimize

# Ground station parameters
min_elevation = theta = np.deg2rad(0)
fov = pi - min_elevation * 2

# Earth parameters
r_E = 6371e3
mu_E = 3.986e14

# Orbital elements
h_orbit = 800e3
a = h_orbit + r_E

## Contact time per pass over Ground Station

In [2]:
# Kepler's Law to find orbital period of circular orbit
T_orbit = 2 * pi * sqrt( a**3 / mu_E )
print("Orbital period in minutes:", T_orbit / 60)


# Earth-centred Angular Distance traversed in one pass - solving a transcendental equation!
def opt_fun(phi, theta, A):
    return ( cos(phi)*(tan(phi)/tan(pi/2 + theta) + 1) - A ) ** 2           # Full derivation in physical notebook...

A = r_E / a
res = minimize(lambda phi: opt_fun(phi, theta, A), x0=0.0001)

# Extract the root from the minimization result
phi = res.x[0]
print("Angular distance traversed in one pass in degrees:", np.rad2deg(phi))


# Time to pass through the ground station
T_pass = T_orbit * phi / (2 * np.pi)
print("Contact time in minutes:", T_pass / 60)


# Maximum distance between mothership and ground station during contact
max_d = a * sin(phi) / sin(pi/2+ theta)
print("Maximum distance between satellite and ground station in km:", max_d/1000)

Orbital period in minutes: 100.72320918114909
Angular distance traversed in one pass in degrees: 27.32155343280554
Contact time in minutes: 7.644207059906707
Maximum distance between satellite and ground station in km: 3291.372834135686


## Link Parameters

#### The link budget is defined as the ratio of the received power to the transmitted power. Should be positive!

The equation for the link budget in dB form is:

### $P_{rx} = P_{tx} + G_{tx} + G_{rx} - L_{tx} - L_{rx} - L_{fs} - L_{misc}$
\
where:

$P_{rx}$ is the received power,\
$P_{tx}$ is the transmitted power,\
$G_{tx}$ is the gain of the transmitter,\
$G_{rx}$ is the gain of the receiver,\
$L_{tx}$ is the loss of the transmitter,\
$L_{rx}$ is the loss of the receiver,\
$L_{fs}$ is the free space path loss, and\
$L_{misc}$ is the loss of miscellaneous factors.

In [3]:
# NOTE - all units are in SI! Could use pint library but I'm lazy

speed_of_light = c = 299792458




### Utility functions ###

# Decibel conversion lambda
dB = lambda x: 10 * np.log10(x)

# Linear conversion lambda
lin = lambda x: 10 ** (x / 10)

### Lambda Functions for Link Budget

In [4]:
# TX Antenna half-power beamwidth - in degrees
# Augumented for f in Hz, D in m
beamwidth = lambda D, f: 21e9 / (f * D)


# Antenna gain in dB - specific to antennas with a circularly symmetric radiating aperture
# Augumented for f in Hz, D in m
G_tx = lambda D, f, eta_tx: 200.4 + 2*dB(D) + 2*dB(f) + dB(eta_tx)     


# Beam shaping - directivity is a function of coverage area in degrees-squared and antenna efficiency
G_tx_shaped = lambda A_theta, eta_tx: 46.15 - dB(A_theta) + dB(eta_tx)


# EIRP - Effective Isotropic Radiated Power = Forward power + Antenna gain
EIRP = lambda P_tx, G_tx: P_tx + G_tx


# Free space path loss - in dB
FSPL = lambda d, f: 2 * ( dB(d) + dB(f) + dB(4*pi / c) )


# Received power - the sum of EIRP and G_rx and minus the FSPL, atmospheric and line losses
C = lambda eirp, G_rx, L_s, L_atm, L_line: eirp + G_rx - L_s - L_atm - L_line


# G/T - Receiver gain to Noise temperature ratio - both in dB
G_T = lambda G_rx, T_rx: G_rx - T_rx


# C/N0 - Received signal power to noise power ratio - both in dB
C_N0 = lambda eirp, g_t, L_total: eirp + g_t - L_total + 228.6


# Eb/N0 - Signal To Noise ratio
Eb_N0 = lambda c_n0, Rb: c_n0 - dB(Rb)

## Sample Links

### Uplink from Ground Station to Mothership

In [6]:
# Set requirements for the link - driven by external systems
data_rate = 1e6                     # 1Mbps = 1e6 bits/s
availability = 0.99                 # 99% availability

# Pointing loss (rolloff in gain) - design point allows for 10% error of the beamwidth
EOC = 3                 # Edge of coverage - arbitarily 3 dB down from peak gain - about 50% loss
L_pointing = 0.12       # Peak pointing loss - in dB - TODO add the EOC error from antenna spec sheet for 10% error


# Modem Implementation Loss
# Required Eb/No
# Link Margin
# Channel Bandwidth
# Number of Channels
# Number of Users/Channel
# Single User Data Rate


x = 524.8074602497722
